In [1]:
# Data import
import pandas as pd
import numpy as np

used_car = pd.read_csv("used_cars.csv")
used_car.head(5)

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,Ford,Utility Police Interceptor Base,2013,"51,000 mi.",E85 Flex Fuel,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capa...,6-Speed A/T,Black,Black,At least 1 accident or damage reported,Yes,"$10,300"
1,Hyundai,Palisade SEL,2021,"34,742 mi.",Gasoline,3.8L V6 24V GDI DOHC,8-Speed Automatic,Moonlight Cloud,Gray,At least 1 accident or damage reported,Yes,"$38,005"
2,Lexus,RX 350 RX 350,2022,"22,372 mi.",Gasoline,3.5 Liter DOHC,Automatic,Blue,Black,None reported,NaN,"$54,598"
3,INFINITI,Q50 Hybrid Sport,2015,"88,900 mi.",Hybrid,354.0HP 3.5L V6 Cylinder Engine Gas/Electric H...,7-Speed A/T,Black,Black,None reported,Yes,"$15,500"
4,Audi,Q3 45 S line Premium Plus,2021,"9,835 mi.",Gasoline,2.0L I4 16V GDI DOHC Turbo,8-Speed Automatic,Glacier White Metallic,Black,None reported,NaN,"$34,999"


First, additional information can be extracted from the dataset. The following steps involve feature engineering, which will be applied before conducting EDA.

In [2]:
# Transform milage and price into numerical features
used_car['milage'] = used_car['milage'].str.replace(' mi.', '').str.replace(',', '').astype(int)
used_car['price'] = used_car['price'].str.replace('$', '').str.replace(',', '').astype('int64')

In [3]:
# Unify nan in fuel_type 
print(used_car["fuel_type"].unique())
used_car['fuel_type'] = used_car['fuel_type'].replace(['–', 'not supported'], np.nan)
print(used_car["fuel_type"].unique())

['E85 Flex Fuel' 'Gasoline' 'Hybrid' nan 'Diesel' 'Plug-In Hybrid' '–'
 'not supported']
['E85 Flex Fuel' 'Gasoline' 'Hybrid' nan 'Diesel' 'Plug-In Hybrid']


In [4]:
len(used_car["engine"].unique())

1146

In [14]:
# Create a new column horsepower by extracting from engine
def extract_horsepower(engine):
    match = pd.Series(engine).str.extract(r'(\d+\.?\d*)HP')
    return match[0] if not match.empty else np.nan

used_car['horsepower'] = used_car['engine'].apply(extract_horsepower).astype(float)

In [15]:
unique_engines = used_car[['engine', 'horsepower']].drop_duplicates()
display(unique_engines)

,engine,horsepower
0,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capa...,300.0
1,3.8L V6 24V GDI DOHC,NaN
2,3.5 Liter DOHC,NaN
3,354.0HP 3.5L V6 Cylinder Engine Gas/Electric H...,354.0
4,2.0L I4 16V GDI DOHC Turbo,NaN
...,...,...
3942,400.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,400.0
3943,211.0HP 2.0L 4 Cylinder Engine Diesel Fuel,211.0
3974,136.0HP 1.8L 4 Cylinder Engine Gasoline Fuel,136.0
3977,270.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,270.0


In [5]:
# Classify Transmission into Automatic, Manual and Other
def classify_transmission(transmission):
    if "manual" in transmission.lower() or "m/t" in transmission.lower():
        return "Manual"
    elif "automatic" in transmission.lower() or "a/t" in transmission.lower():
        return "Automatic"
    else:
        return "Unknown"

used_car['transmission_type'] = used_car['transmission'].apply(classify_transmission)

In [10]:
# Create a new column num_gears by extracting the number of gears from transmission
def extract_gears(transmission):
    match = pd.Series(transmission).str.extract(r'(\d+)')
    return match[0] if not match.empty else np.nan

used_car['num_gears'] = used_car['transmission'].apply(extract_gears).astype(float)

In [11]:
# Check the transmission_type, and num_gears of each unique transformation
# pd.set_option('display.max_rows', None)
unique_transmissions = used_car[['transmission', 'transmission_type', 'num_gears']].drop_duplicates()
display(unique_transmissions)

,transmission,transmission_type,num_gears
0,6-Speed A/T,Automatic,6.0
1,8-Speed Automatic,Automatic,8.0
2,Automatic,Automatic,NaN
3,7-Speed A/T,Automatic,7.0
5,F,Unknown,NaN
...,...,...,...
2393,9-Speed Automatic with Auto-Shift,Automatic,9.0
2620,SCHEDULED FOR OR IN PRODUCTION,Unknown,NaN
2772,2-Speed A/T,Automatic,2.0
3206,6 Speed Mt,Unknown,6.0


In [8]:
for col in used_car.columns:
    print(f"Column: {col}, Data type: {used_car[col].dtype}")

Column: brand, Data type: object
Column: model, Data type: object
Column: model_year, Data type: int64
Column: milage, Data type: int64
Column: fuel_type, Data type: object
Column: engine, Data type: object
Column: transmission, Data type: object
Column: ext_col, Data type: object
Column: int_col, Data type: object
Column: accident, Data type: object
Column: clean_title, Data type: object
Column: price, Data type: int64
Column: transmission_type, Data type: object
Column: num_gears, Data type: float64
